In [1]:
%matplotlib inline
import matplotlib.pyplot as plt
import IPython.display as ipd

import os
import json
import math
import torch
from torch import nn
from torch.nn import functional as F
from torch.utils.data import DataLoader

import commons
import utils
from data_utils import (
    TextAudioLoader,
    TextAudioCollate,
    TextAudioSpeakerLoader,
    TextAudioSpeakerCollate,
)
from models import SynthesizerTrn
from text.symbols import symbols
from text import text_to_sequence

from scipy.io.wavfile import write


def get_text(text, hps):
    text_norm = text_to_sequence(text, hps.data.text_cleaners)
    if hps.data.add_blank:
        text_norm = commons.intersperse(text_norm, 0)
    text_norm = torch.LongTensor(text_norm)
    return text_norm

DEBUG:numba.core.byteflow:bytecode dump:
>          0	NOP(arg=None, lineno=1039)
           2	RESUME(arg=0, lineno=1039)
           4	LOAD_FAST(arg=0, lineno=1042)
           6	LOAD_CONST(arg=1, lineno=1042)
           8	BINARY_SUBSCR(arg=None, lineno=1042)
          18	LOAD_FAST(arg=0, lineno=1042)
          20	LOAD_CONST(arg=2, lineno=1042)
          22	BINARY_SUBSCR(arg=None, lineno=1042)
          32	COMPARE_OP(arg=4, lineno=1042)
          38	LOAD_FAST(arg=0, lineno=1042)
          40	LOAD_CONST(arg=1, lineno=1042)
          42	BINARY_SUBSCR(arg=None, lineno=1042)
          52	LOAD_FAST(arg=0, lineno=1042)
          54	LOAD_CONST(arg=3, lineno=1042)
          56	BINARY_SUBSCR(arg=None, lineno=1042)
          66	COMPARE_OP(arg=5, lineno=1042)
          72	BINARY_OP(arg=1, lineno=1042)
          76	RETURN_VALUE(arg=None, lineno=1042)
DEBUG:numba.core.byteflow:pending: deque([State(pc_initial=0 nstack_initial=0)])
DEBUG:numba.core.byteflow:stack: []
DEBUG:numba.core.byteflow:state.pc

In [2]:
device = torch.device("cpu")

# LJSpeech

In [3]:
hps = utils.get_hparams_from_file("logs/ljs_base/config.json")

In [4]:
net_g = SynthesizerTrn(
    len(symbols),
    hps.data.filter_length // 2 + 1,
    hps.train.segment_size // hps.data.hop_length,
    **hps.model).cuda()
_ = net_g.eval()

_ = utils.load_checkpoint("logs/ljs_base/G_3000.pth", net_g, None)

RuntimeError: Error(s) in loading state_dict for SynthesizerTrn:
	size mismatch for enc_p.emb.weight: copying a param with shape torch.Size([178, 192]) from checkpoint, the shape in current model is torch.Size([124, 192]).
	size mismatch for enc_q.pre.weight: copying a param with shape torch.Size([192, 80, 1]) from checkpoint, the shape in current model is torch.Size([192, 513, 1]).

In [ ]:
stn_tst = get_text("VITS is Awesome!", hps)
with torch.no_grad():
    x_tst = stn_tst.cuda().unsqueeze(0)
    x_tst_lengths = torch.LongTensor([stn_tst.size(0)]).cuda()
    audio = net_g.infer(x_tst, x_tst_lengths, noise_scale=.667, noise_scale_w=0.8, length_scale=1)[0][0,0].data.cpu().float().numpy()
ipd.display(ipd.Audio(audio, rate=hps.data.sampling_rate, normalize=False))

# VCTK

In [ ]:
hps = utils.get_hparams_from_file("./configs/vits2_vctk_base2.json")

In [ ]:
if hps.model.use_mel_posterior_encoder == True:
    print("Using mel posterior encoder for VITS2")
    posterior_channels = 80  # vits2
    hps.data.use_mel_posterior_encoder = True
else:
    print("Using lin posterior encoder for VITS1")
    posterior_channels = hps.data.filter_length // 2 + 1
    hps.data.use_mel_posterior_encoder = False

net_g = SynthesizerTrn(
    len(symbols),
    hps.data.n_mel_channels,
    None,
    n_speakers=hps.data.n_speakers,
    **hps.model,
).to(device)
_ = net_g.eval()

_ = utils.load_checkpoint("/path/to/the/pretrained.pth", net_g, None)

In [ ]:
text = """VITS2 is Awesome!"""
sid = 4

stn_tst = get_text(text, hps)
with torch.no_grad():
    x_tst = stn_tst.to(device).unsqueeze(0)
    x_tst_lengths = torch.LongTensor([stn_tst.size(0)]).to(device)
    sid = torch.LongTensor([int(sid)]).to(device)
    audio = (
        net_g.infer(
            x_tst,
            x_tst_lengths,
            sid=sid,
            noise_scale=0.667,
            noise_scale_w=0.8,
            length_scale=1,
        )[0][0, 0]
        .data.cpu()
        .float()
        .numpy()
    )
ipd.display(ipd.Audio(audio, rate=hps.data.sampling_rate, normalize=False))

# Voice Conversion

In [ ]:
dataset = TextAudioSpeakerLoader(hps.data.validation_files, hps.data)
collate_fn = TextAudioSpeakerCollate()
loader = DataLoader(dataset, num_workers=0, shuffle=False,
    batch_size=1, pin_memory=False,
    drop_last=True, collate_fn=collate_fn)
data_list = list(loader)

In [ ]:
with torch.no_grad():
    x, x_lengths, spec, spec_lengths, y, y_lengths, sid_src = [x.to(device) for x in data_list[0]]
    sid_tgt1 = torch.LongTensor([1]).to(device)
    sid_tgt2 = torch.LongTensor([2]).to(device)
    sid_tgt3 = torch.LongTensor([4]).to(device)
    audio1 = net_g.voice_conversion(spec, spec_lengths, sid_src=sid_src, sid_tgt=sid_tgt1)[0][0,0].data.cpu().float().numpy()
    audio2 = net_g.voice_conversion(spec, spec_lengths, sid_src=sid_src, sid_tgt=sid_tgt2)[0][0,0].data.cpu().float().numpy()
    audio3 = net_g.voice_conversion(spec, spec_lengths, sid_src=sid_src, sid_tgt=sid_tgt3)[0][0,0].data.cpu().float().numpy()
print("Original SID: %d" % sid_src.item())
ipd.display(ipd.Audio(y[0].cpu().numpy(), rate=hps.data.sampling_rate, normalize=False))
print("Converted SID: %d" % sid_tgt1.item())
ipd.display(ipd.Audio(audio1, rate=hps.data.sampling_rate, normalize=False))
print("Converted SID: %d" % sid_tgt2.item())
ipd.display(ipd.Audio(audio2, rate=hps.data.sampling_rate, normalize=False))
print("Converted SID: %d" % sid_tgt3.item())
ipd.display(ipd.Audio(audio3, rate=hps.data.sampling_rate, normalize=False))